PYTHON

In [10]:
import pandas as pd
from datetime import datetime
from privatefunctions import add6ColFile, add8ColFile, add12ColFile, add14ColFile

In [11]:
## Git clonning and pulling:

print('***', 'GETTING FILES FROM Johns Hopkins University ...')

!git clone https://github.com/CSSEGISandData/COVID-19
%cd './COVID-19'
!git pull
!ls
%cd ..
!ls

*** GETTING FILES FROM Johns Hopkins University ...
fatal: destination path 'COVID-19' already exists and is not an empty directory.
/Users/adrianosastre/datascience-covid-19/COVID-19
Already up to date.
README.md                      csse_covid_19_data
archived_data                  who_covid_19_situation_reports
/Users/adrianosastre/datascience-covid-19
COVID-19               covid-19.ipynb         standardplot.py
Pandas_Cheat_Sheet.pdf hello.py
__pycache__            privatefunctions.py


In [12]:
## Getting csv data:
daily_reports_path = './COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/'
csv_file_names = !ls $daily_reports_path | grep .csv

print('***', len(csv_file_names), 'DAILY FILES IN Johns Hopkins University.')

*** 144 DAILY FILES IN Johns Hopkins University.


In [14]:
main_df = pd.DataFrame(
  columns=[
    'Date', 
    'City', 
    'State', 
    'Country', 
    'Latitude', 
    'Longitude', 
    'TotalConfirmed', 
    'TotalDeaths', 
    'TotalRecovered', 
    'TotalActive', 
    'CaseFatalityRatio', 
    'IncidenceRate', 
    'MortalityRate',
    'DailyConfirmed', 
    'DailyDeaths', 
    'DailyRecovered', 
    'DailyActive']) 

print('*** INITIAL MAIN DATAFRAME: ')

print(main_df)
print(main_df.dtypes)

*** INITIAL MAIN DATAFRAME: 
Empty DataFrame
Columns: [Date, City, State, Country, Latitude, Longitude, TotalConfirmed, TotalDeaths, TotalRecovered, TotalActive, CaseFatalityRatio, IncidenceRate, MortalityRate, DailyConfirmed, DailyDeaths, DailyRecovered, DailyActive]
Index: []
Date                 object
City                 object
State                object
Country              object
Latitude             object
Longitude            object
TotalConfirmed       object
TotalDeaths          object
TotalRecovered       object
TotalActive          object
CaseFatalityRatio    object
IncidenceRate        object
MortalityRate        object
DailyConfirmed       object
DailyDeaths          object
DailyRecovered       object
DailyActive          object
dtype: object


In [13]:
print('*** WILL START GETTING DATA FROM Johns Hopkins University AND NORMALIZE COLUMNS NAMES: ')

# 1 file per day:
for file in csv_file_names:

  file_path = daily_reports_path + file

  # read daily csv data:
  df = pd.read_csv(daily_reports_path + file)

  # get date from file name:
  date = datetime.strptime(file[0:-4], '%m-%d-%Y').date()

  print('*** DATE:', date, df.shape[0], 'lines,', df.shape[1], 'columns ...')

  # 6-columns file format:
  if df.shape[1] == 6 : 
    main_df = add6ColFile(date, df, main_df)

  # 8-columns file format:
  elif df.shape[1] == 8 :
    main_df = add8ColFile(date, df, main_df)

  # 12-columns file format:
  elif df.shape[1] == 12 :
    main_df = add12ColFile(date, df, main_df)
    
  # 14-columns file format:
  elif df.shape[1] == 14 :
    main_df = add14ColFile(date, df, main_df)

print('*** MAIN DATAFRAME AFTER GETTING DATA FROM Johns Hopkins University: ')

print(main_df.shape)
print(main_df.dtypes)
print(main_df)
print(main_df.dtypes)
print(main_df.shape)

*** WILL START GETTING DATA FROM Johns Hopkins University AND NORMALIZE COLUMNS NAMES: 
*** DATE: 2020-01-22 38 lines, 6 columns ...


NameError: name 'main_df' is not defined

In [ ]:
print('*** WILL CALCULATE DAILY NUMBERS ... ')

previousDayConfirmed = 0
previousDayDeaths = 0
previousDayRecovered = 0
previousDayActive = 0

for i in range(len(main_df)):

    print('*** DATE:', main_df.iloc[i,'Date'], '...')

    totalConfirmed = main_df.iloc[i,'TotalConfirmed']
    main_df.iloc[i,'DailyConfirmed'] = totalConfirmed - previousDayConfirmed
    previousDayConfirmed = totalConfirmed

    totalDeaths = main_df.iloc[i,'TotalDeaths']
    main_df.iloc[i,'DailyDeaths'] = totalDeaths - previousDayDeaths
    previousDayDeths = totalDeaths

    totalRecovered = main_df.iloc[i,'TotalRecovered']
    main_df.iloc[i,'DailyRecovered'] = totalRecovered - previousDayRecovered
    previousDayRecovered = totalRecovered

    totalActive = main_df.iloc[i,'TotalActive']
    main_df.iloc[i,'DailyActive'] = totalActive - previousDayActive
    previousDayActive = totalActive

print('*** MAIN DATAFRAME AFTER CALCULATING DAILY NUMBERS: ')

print(main_df.shape)
print(main_df.dtypes)
print(main_df)
print(main_df.dtypes)
print(main_df.shape)
